`otto` automatically tests functions and classes.  Defining classes should do more than change the namespace.

    %reload_ext XXX.otto
    
to install automatic testing.

In [1]:
from unittest import *
from doctest import *
from ast import *

from typing import Callable
from dataclasses import dataclass, field

from inspect import signature

In [12]:
    @dataclass
    class Testing(object):
        shell: 'ip' = field(default_factory=get_ipython)
        tests = list()
        
        def pre_execute(self): Testing.tests = []
            
        def post_run_cell(Testing):
            from types import ModuleType
            globs = {}
            tests = list()
            module, main = ModuleType('__main__'), __import__('__main__')
            module.__test__ = {}
            if not Testing.tests: return
            while Testing.tests:
                object = Testing.tests.pop(0)
                current = str(len(module.__test__))
                if isinstance(object, Str):
                    module.__test__[current] = object.s
                if isinstance(object, ClassDef):    
                    object = getattr(main, object.name)
                    if hasattr(object, 'runTest') and not issubclass(object, TestCase):
                        object = type(object.__name__, (TestCase, object), {})
                    if issubclass(object, TestCase):
                        tests.append(object())
                    module.__test__[current] = object
                if isinstance(object, FunctionDef):
                    object = getattr(main, object.name)
                    try:
                        if not bool(dict(signature(object).parameters)):
                            tests.append(FunctionTestCase(object))
                    except: ...
                    if getattr(object, '__doc__', ''):
                        module.__test__[object.__name__] = object
            if tests or module.__test__:
                tests.append(DocTestSuite(module, vars(main)))
                suite = TestSuite(tests)
                TextTestRunner().run(suite)

In [13]:
    class DiscoverTests(NodeTransformer):
        def visit_star(self, node):
            Testing.tests += node,
            return node
        visit_Str = visit_ClassDef =  visit_FunctionDef = visit_star

In [14]:
    def load_ipython_extension(ip=get_ipython()):
        testing = Testing(shell=get_ipython())
        ip.ast_transformers = [DiscoverTests()]
        ip.events.register('pre_execute', testing.pre_execute)
        ip.events.register('post_run_cell', testing.post_run_cell)